# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [102]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [103]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
'''
with open ('anna.txt', 'r') as f:
    text = f.read()

vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
'''

"\nwith open ('anna.txt', 'r') as f:\n    text = f.read()\n\nvocab = sorted(set(text))\nvocab_to_int = {c: i for i, c in enumerate(vocab)}\nint_to_vocab = dict(enumerate(vocab))\nencoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)\n"

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [104]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [105]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [106]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [107]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # n_seqs 만큼 batch에서 이동을 한다.
    # Get the number of characters per batch and number of batches we can make

    batch_size = n_seqs * n_steps
    print('batch_size: ', batch_size)
    print('len(arr): ', len(arr))
    n_batches = len(arr) // batch_size
    print('n_batchs: ', n_batches)
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    print('len(arr.reshape): ', len(arr))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [108]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

batch_size:  500
len(arr):  1985223
n_batchs:  3970
len(arr.reshape):  10


In [109]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [110]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [111]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [112]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predicts')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [113]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =  tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [114]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [115]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [123]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [124]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

batch_size:  10000
len(arr):  1985223
n_batchs:  198
len(arr.reshape):  100
Epoch: 1/20...  Training Step: 1...  Training loss: 4.4169...  2.4275 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3133...  2.3693 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.7296...  2.4255 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 4.8900...  2.3432 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.2089...  2.3011 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.9403...  2.3292 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.7794...  2.3051 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.6426...  2.3272 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.4832...  2.3292 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.4666...  2.3352 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.4125...  2.2981 sec/batch
Epoch: 1/20...  Training Step: 12...  Trai

Epoch: 1/20...  Training Step: 102...  Training loss: 3.1074...  2.3533 sec/batch
Epoch: 1/20...  Training Step: 103...  Training loss: 3.1068...  2.3663 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.0982...  2.4134 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.0962...  2.4014 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.0906...  2.3392 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0713...  2.3352 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0800...  2.3553 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0877...  2.3132 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.0390...  2.3051 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 3.0596...  2.3563 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.0619...  2.3362 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 3.0512...  2.3021 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 201...  Training loss: 2.4675...  2.2941 sec/batch
Epoch: 2/20...  Training Step: 202...  Training loss: 2.4797...  2.3101 sec/batch
Epoch: 2/20...  Training Step: 203...  Training loss: 2.4792...  2.2951 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.4726...  2.2941 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.4807...  2.3031 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.4750...  2.3563 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.4938...  2.3613 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.4678...  2.3101 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.4501...  2.2921 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.4727...  2.3041 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.4671...  2.2961 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.4948...  2.3011 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 301...  Training loss: 2.2310...  2.3503 sec/batch
Epoch: 2/20...  Training Step: 302...  Training loss: 2.2435...  2.3733 sec/batch
Epoch: 2/20...  Training Step: 303...  Training loss: 2.2254...  2.3111 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.2414...  2.2971 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.2331...  2.2961 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.2607...  2.2921 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.2525...  2.3081 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.2231...  2.3432 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.2423...  2.3543 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.2435...  2.3051 sec/batch
Epoch: 2/20...  Training Step: 311...  Training loss: 2.2285...  2.2881 sec/batch
Epoch: 2/20...  Training Step: 312...  Training loss: 2.2186...  2.2931 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 400...  Training loss: 2.0808...  2.3001 sec/batch
Epoch: 3/20...  Training Step: 401...  Training loss: 2.0913...  2.2941 sec/batch
Epoch: 3/20...  Training Step: 402...  Training loss: 2.0637...  2.2771 sec/batch
Epoch: 3/20...  Training Step: 403...  Training loss: 2.0918...  2.3603 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.0867...  2.3432 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.1197...  2.3212 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.0788...  2.3041 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.0662...  2.3232 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.0593...  2.2971 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 2.0799...  2.2901 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 2.1070...  2.3633 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 2.0610...  2.3332 sec/batch
Epoch: 3/20...  

...

Epoch: 18/20...  Training Step: 3542...  Training loss: 1.2185...  2.2941 sec/batch
Epoch: 18/20...  Training Step: 3543...  Training loss: 1.2375...  2.2971 sec/batch
Epoch: 18/20...  Training Step: 3544...  Training loss: 1.2766...  2.3583 sec/batch
Epoch: 18/20...  Training Step: 3545...  Training loss: 1.2263...  2.3482 sec/batch
Epoch: 18/20...  Training Step: 3546...  Training loss: 1.2123...  2.3031 sec/batch
Epoch: 18/20...  Training Step: 3547...  Training loss: 1.2232...  2.3021 sec/batch
Epoch: 18/20...  Training Step: 3548...  Training loss: 1.2411...  2.3091 sec/batch
Epoch: 18/20...  Training Step: 3549...  Training loss: 1.2343...  2.2931 sec/batch
Epoch: 18/20...  Training Step: 3550...  Training loss: 1.2285...  2.2921 sec/batch
Epoch: 18/20...  Training Step: 3551...  Training loss: 1.2270...  2.3242 sec/batch
Epoch: 18/20...  Training Step: 3552...  Training loss: 1.2234...  2.3442 sec/batch
Epoch: 18/20...  Training Step: 3553...  Training loss: 1.2687...  2.2831 se

Epoch: 19/20...  Training Step: 3639...  Training loss: 1.2351...  2.3442 sec/batch
Epoch: 19/20...  Training Step: 3640...  Training loss: 1.2451...  2.3392 sec/batch
Epoch: 19/20...  Training Step: 3641...  Training loss: 1.2306...  2.2841 sec/batch
Epoch: 19/20...  Training Step: 3642...  Training loss: 1.2151...  2.2791 sec/batch
Epoch: 19/20...  Training Step: 3643...  Training loss: 1.2039...  2.2951 sec/batch
Epoch: 19/20...  Training Step: 3644...  Training loss: 1.2315...  2.2891 sec/batch
Epoch: 19/20...  Training Step: 3645...  Training loss: 1.2029...  2.2981 sec/batch
Epoch: 19/20...  Training Step: 3646...  Training loss: 1.2316...  2.3392 sec/batch
Epoch: 19/20...  Training Step: 3647...  Training loss: 1.2037...  2.3613 sec/batch
Epoch: 19/20...  Training Step: 3648...  Training loss: 1.2248...  2.2841 sec/batch
Epoch: 19/20...  Training Step: 3649...  Training loss: 1.2078...  2.2931 sec/batch
Epoch: 19/20...  Training Step: 3650...  Training loss: 1.2293...  2.3061 se

Epoch: 19/20...  Training Step: 3737...  Training loss: 1.2603...  2.2961 sec/batch
Epoch: 19/20...  Training Step: 3738...  Training loss: 1.2311...  2.3111 sec/batch
Epoch: 19/20...  Training Step: 3739...  Training loss: 1.2319...  2.2781 sec/batch
Epoch: 19/20...  Training Step: 3740...  Training loss: 1.2040...  2.2841 sec/batch
Epoch: 19/20...  Training Step: 3741...  Training loss: 1.2284...  2.3392 sec/batch
Epoch: 19/20...  Training Step: 3742...  Training loss: 1.2735...  2.3422 sec/batch
Epoch: 19/20...  Training Step: 3743...  Training loss: 1.2066...  2.2901 sec/batch
Epoch: 19/20...  Training Step: 3744...  Training loss: 1.2167...  2.2871 sec/batch
Epoch: 19/20...  Training Step: 3745...  Training loss: 1.2085...  2.3021 sec/batch
Epoch: 19/20...  Training Step: 3746...  Training loss: 1.2249...  2.2871 sec/batch
Epoch: 19/20...  Training Step: 3747...  Training loss: 1.2318...  2.2831 sec/batch
Epoch: 19/20...  Training Step: 3748...  Training loss: 1.2116...  2.3302 se

Epoch: 20/20...  Training Step: 3834...  Training loss: 1.2458...  2.2981 sec/batch
Epoch: 20/20...  Training Step: 3835...  Training loss: 1.2561...  2.2971 sec/batch
Epoch: 20/20...  Training Step: 3836...  Training loss: 1.2116...  2.3523 sec/batch
Epoch: 20/20...  Training Step: 3837...  Training loss: 1.2283...  2.3232 sec/batch
Epoch: 20/20...  Training Step: 3838...  Training loss: 1.2397...  2.2951 sec/batch
Epoch: 20/20...  Training Step: 3839...  Training loss: 1.2215...  2.2921 sec/batch
Epoch: 20/20...  Training Step: 3840...  Training loss: 1.2086...  2.2961 sec/batch
Epoch: 20/20...  Training Step: 3841...  Training loss: 1.1889...  2.2991 sec/batch
Epoch: 20/20...  Training Step: 3842...  Training loss: 1.2414...  2.3613 sec/batch
Epoch: 20/20...  Training Step: 3843...  Training loss: 1.1907...  2.3412 sec/batch
Epoch: 20/20...  Training Step: 3844...  Training loss: 1.2161...  2.3513 sec/batch
Epoch: 20/20...  Training Step: 3845...  Training loss: 1.2017...  2.2831 se

Epoch: 20/20...  Training Step: 3932...  Training loss: 1.2202...  2.3302 sec/batch
Epoch: 20/20...  Training Step: 3933...  Training loss: 1.2163...  2.3041 sec/batch
Epoch: 20/20...  Training Step: 3934...  Training loss: 1.2098...  2.2971 sec/batch
Epoch: 20/20...  Training Step: 3935...  Training loss: 1.2518...  2.2911 sec/batch
Epoch: 20/20...  Training Step: 3936...  Training loss: 1.2158...  2.2931 sec/batch
Epoch: 20/20...  Training Step: 3937...  Training loss: 1.2247...  2.2851 sec/batch
Epoch: 20/20...  Training Step: 3938...  Training loss: 1.1944...  2.3392 sec/batch
Epoch: 20/20...  Training Step: 3939...  Training loss: 1.2233...  2.3392 sec/batch
Epoch: 20/20...  Training Step: 3940...  Training loss: 1.2581...  2.3001 sec/batch
Epoch: 20/20...  Training Step: 3941...  Training loss: 1.2066...  2.2981 sec/batch
Epoch: 20/20...  Training Step: 3942...  Training loss: 1.1988...  2.2991 sec/batch
Epoch: 20/20...  Training Step: 3943...  Training loss: 1.2085...  2.2941 se

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [125]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200_l512.ckpt"
all_mo

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [126]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [127]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [128]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i3960_l512.ckpt'

In [129]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i3960_l512.ckpt
Farrand with sereness of the same the
sight of the porter, so so that he had no circle to hild. The party.
The consideration of the prayer of her force himself, and would have so taken
ut the sight of the prince's position, an intellectual form were not
lively, wore in sheeps, a consideration with him, what was hated in her
sister without him. That this, and the corner of the conversation that it
was the season of the service in which the chief cold bother, and she was
that the same interests with a convinced of her position in the significance
of the marsh had been a grut head of such answering, and to be the more only
there would not be so time and set in a bear in the conders, but he had to
send the porter, but there are anyone whee she seemed, the sort of tract
in the priest and the same stands, and the man's sort of service, because
it was shame to see anything, and that she could not come to him over his mother

In [130]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Fard." "
"We lad onder atos whe se hers on he har seod shethe ho wher tore tingend wong tis ore and and are ales tor the the wan one shad tas the sate hit he shas hite wint, he wot an ases ore to sist at het orer sisserare wo wh he ane arisighe and thir ones af at and on at ou hed she sones sesing ho to hhis ar the sot hhad wase thar ass here sithend wat he ter and onte hasd, wan athen ar tal ath on and torere son thes tand,
tadd him he whir ous and to he he alde the sor tin the ass at tass tom
southind, wis toten the herad tho sistin hit he sees and sase the he hes an ans ood the sereditig this har there anthe hans, sothe whis and to wat the wit has the sot ou the sissing thes handend weresen tas sete sh mos onther ante she an sos atito hha wherse sesse sad wiss hot at hes oo sess and tit sissoresin this alled on the wint oo harit hit alin tand, and hes atise sithe wor hes tomint, wos sot he were sone wan sate worese

In [131]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Farring, haid,
and whine was now allows
and himsens the prosces, which
this say the carlong as him her had have
take her
wheles of
shimess, shaid of chele and and the sach of her hors, to diven to his souned hissely that the camt too and to as has her was his brough she sand as there would not to the mond that he weling all the hurd of her the must thay shis she had not said, and hord of her
was of her sainess, and wine the
marsing to hived to
the wert one
comeranite. "You was then the
souted in his all say...

"Whe colling the tore and were, was
serited to ham.

"I wand trook at antiththan to she to sher the ceratane. "I't to grink of the comt on the pain a pise to the mongrom.

"In what ha sid to the wase, but she sant her alexed some and thous shouse and thought hav into his sout, and was then
she tas say the fisticiant to all she had strook was
out, anowhen the conster of she had him seetend the minch, his bughere

In [132]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Farremed to the
sorn of his coming the
tabe, and same of the prancess that he was never to be seet
at the chulconess and and, at stalled as any
poored of the send of cares. And
the casion, said, and was not that that there had seemed
her husband, there had now
their the tristen, and thinker on the partered as the serventer, that him to the stile went and a sunder, shoulding the sore of horrial and husband that
was simply she wanted, who had some much she was till to seem he
had been both to be said the disprince all his sair, he can to him. How had been too much
any answorat were she
could said her. He had not a must
see he saw the clear, and the conscition if they. He went to be and taking and he wanced into she had been and talked in the pirters. And he had not happened, brugh taking only to the stood of the partion of this hand of
him at his head that she seat to give her too.

"Why dressed you thene way it all, I